In [3]:
# -*- coding:utf-8 -*-

from sklearn.feature_extraction.text import CountVectorizer
import os
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
from sklearn import svm
from sklearn.feature_extraction.text import TfidfTransformer
import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, global_max_pool
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
from tflearn.data_utils import to_categorical, pad_sequences
from sklearn.neural_network import MLPClassifier
from tflearn.layers.normalization import local_response_normalization
# from tensorflow.contrib import learn
import gensim
import re
from collections import namedtuple
from gensim.models import Doc2Vec
# from gensim.models.doc2vec import Doc2Vec,LabeledSentence
from random import shuffle
import multiprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import scale
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn import preprocessing
from hmmlearn import hmm
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
from tflearn.layers.recurrent import bidirectional_rnn, BasicLSTMCell

# 14.1　数据集

In [ ]:
def load_files_from_dir(dir):
    import glob
    files=glob.glob(dir)
    result = []
    for file in files:
        #print("Load file %s" % file
        with open(file) as f:
            lines=f.readlines()
            lines_to_line=" ".join(lines)
            lines_to_line = re.sub(r"[APT|Crypto|Locker|Zeus]", ' ', lines_to_line,flags=re.I)
            result.append(lines_to_line)
    return result

In [ ]:
def load_files():
    #/data/malware/MalwareTrainingSets-master/trainingSets
    malware_class=['APT1','Crypto','Locker','Zeus']
    x=[]
    y=[]
    for i,family in enumerate(malware_class):
        dir="../Data/malware/MalwareTrainingSets-master/trainingSets/%s/*" % family
        print("Load files from %s index %d" % (dir,i))
        v=load_files_from_dir(dir)
        x+=v
        y+=[i]*len(v)
    print("Loaded files %d" % len(x))
    return x,y

def get_feature_pe_picture():
    #加载原始文件
    x,y=load_files()
    max_features=1024
    vectorizer = CountVectorizer(
            decode_error='ignore',
            ngram_range=(2, 2),
            strip_accents='ascii',
            max_features=max_features,
            stop_words='english',
            max_df=1.0,
            min_df=1,
            dtype=np.int,
            token_pattern=r'\b\w+\b',
            binary=False)
    # print vectorizer
    x=vectorizer.fit_transform(x)
    #非常重要 稀疏矩阵转换成矩阵
    x=x.toarray()
    x_pic = []
    for i in range(4762):
        #将形状为(1024，1)的向量转化成（32，32）的矩阵
        pic=np.reshape(x[i],(32,32,1))
        x_pic.append(pic)
        #save_image(pic,i)
    #随机分配训练和测试集合
    x_train, x_test, y_train, y_test = train_test_split(x_pic, y, test_size=0.4)
    return x_train, x_test, y_train, y_test


In [ ]:

def do_xgboost(x_train, x_test, y_train, y_test):
    xgb_model = xgb.XGBClassifier().fit(x_train, y_train)
    y_pred = xgb_model.predict(x_test)
    print(classification_report(y_test, y_pred))

def do_svm(x_train, x_test, y_train, y_test):
    from sklearn.svm import SVC
    clf = svm.SVC(kernel='linear', C=1.0)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(classification_report(y_test, y_pred))

def do_mlp(x_train, x_test, y_train, y_test):

    clf = MLPClassifier(solver='lbfgs',
                        alpha=1e-5,
                        hidden_layer_sizes = (10, 4),
                        random_state = 1)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(classification_report(y_test, y_pred))
    #print metrics.confusion_matrix(y_test, y_pred)

def get_feature_text():
    x,y=load_files()
    max_features=1000

    vectorizer = CountVectorizer(
            decode_error='ignore',
            ngram_range=(2, 2),
            strip_accents='ascii',
            max_features=max_features,
            stop_words='english',
            max_df=1.0,
            min_df=1,
            token_pattern=r'\b\w+\b',
            binary=True)
    # print vectorizer
    x=vectorizer.fit_transform(x)

    transformer = TfidfTransformer(smooth_idf=False)
    x = transformer.fit_transform(x)

    # 非常重要 稀疏矩阵转换成矩阵
    x = x.toarray()

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)
    return x_train, x_test, y_train, y_test

def save_image(data,index):
    from PIL import Image
    new_im = Image.fromarray(data)
    new_im.save("../Data/malware/%s.bmp" % index)



def do_cnn(trainX, testX, trainY, testY):
    # Converting labels to binary vectors
    trainY = to_categorical(trainY, nb_classes=4)
    testY = to_categorical(testY, nb_classes=4)
    # Building convolutional network
    network = input_data(shape=[None, 32, 32,1], name='input')
    network = conv_2d(network, 16, 3, activation='relu', regularizer="L2")
    network = max_pool_2d(network, 2)
    network = local_response_normalization(network)
    network = conv_2d(network, 16, 3, activation='relu', regularizer="L2")
    network = max_pool_2d(network, 2)
    network = local_response_normalization(network)
    network = fully_connected(network, 16, activation='tanh')
    network = dropout(network, 0.1)
    network = fully_connected(network, 16, activation='tanh')
    network = dropout(network, 0.1)
    network = fully_connected(network, 4, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=0.01,
                         loss='categorical_crossentropy', name='target')

    # Training
    model = tflearn.DNN(network, tensorboard_verbose=0)
    model.fit(trainX, trainY, n_epoch=10, validation_set=(testX, testY),show_metric=True, run_id="malware")

def do_cnn_1d(trainX, testX, trainY, testY):

    # Converting labels to binary vectors
    trainY = to_categorical(trainY, nb_classes=4)
    testY = to_categorical(testY, nb_classes=4)

    # Building convolutional network
    network = input_data(shape=[None,1000], name='input')
    network = tflearn.embedding(network, input_dim=1000000, output_dim=128,validate_indices=False)
    branch1 = conv_1d(network, 128, 3, padding='valid', activation='relu', regularizer="L2")
    branch2 = conv_1d(network, 128, 4, padding='valid', activation='relu', regularizer="L2")
    branch3 = conv_1d(network, 128, 5, padding='valid', activation='relu', regularizer="L2")
    network = merge([branch1, branch2, branch3], mode='concat', axis=1)
    network = tf.expand_dims(network, 2)
    network = global_max_pool(network)
    network = dropout(network, 0.8)
    network = fully_connected(network, 4, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=0.001,
                         loss='categorical_crossentropy', name='target')
    # Training
    model = tflearn.DNN(network, tensorboard_verbose=0)
    model.fit(trainX, trainY,
              n_epoch=5, shuffle=True, validation_set=(testX, testY),
              show_metric=True, batch_size=100,run_id="malware")

if __name__ == "__main__":
    # print("Hello malware"

    # print("text feature and cnn"
    x_train, x_test, y_train, y_test=get_feature_text()
    do_cnn_1d(x_train, x_test, y_train, y_test)


"""
    print("text feature and xgboost"
    x_train, x_test, y_train, y_test=get_feature_text()
    do_xgboost(x_train, x_test, y_train, y_test)

    print("text feature and svm"
    x_train, x_test, y_train, y_test=get_feature_text()
    do_svm(x_train, x_test, y_train, y_test)

    print("text feature and mlp"
    x_train, x_test, y_train, y_test=get_feature_text()
    do_mlp(x_train, x_test, y_train, y_test)
"""